In [1]:
MONGO_URI = ""

## 1. Vanilla Vector Search

### 1.1 Data loading

In [2]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset('csv', data_files='./data/recipes/recipes.csv')

/Users/alexis/Projects/202407 RAG/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Convert the dataset to a pandas dataframe
df_dataset = dataset['train'].to_pandas()

# drop duplicates
df_dataset = df_dataset = df_dataset.drop(columns='id').drop_duplicates()

df_dataset.head(3)

,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,Apple-Cranberry Crostada,None,None,None,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,None,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...


In [4]:
### generates embeddings 

from openai import OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def get_embedding(text, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

print(get_embedding("Once upon a time, there was a cat."))

[0.032019928097724915, 0.06339229643344879, -0.14854365587234497, -0.0749576985836029, 0.05321422964334488, 0.0338364876806736, -0.08441770821809769, 0.040955618023872375, -0.022981032729148865, 0.006492594722658396, -0.002992316149175167, 0.03711792826652527, 0.07530311495065689, 0.03950294107198715, -0.07141084969043732, -0.07568350434303284, 0.03580918163061142, -0.035198185592889786, 0.023276833817362785, 0.05217314511537552, -0.0002876944199670106, -0.03401849791407585, 0.06201925501227379, 0.027593346312642097, 0.03676736727356911, 0.05155615136027336, -0.047059908509254456, 0.04728643223643303, -0.013932494446635246, 0.011554342694580555, 0.019040672108530998, -0.025935795158147812, 0.006764427293092012, -0.04029737412929535, -0.016108892858028412, -0.03797941654920578, 0.07615499198436737, 0.014955293387174606, 0.04871508851647377, 0.057611603289842606, 0.028666773810982704, 0.038383256644010544, -0.041735850274562836, -0.00832691416144371, 0.049295950680971146, 0.0231716521084

In [5]:
df_dataset['text_embeddings'] = None

for i, row in df_dataset.iterrows():
    text = row['recipe_name'] + " " + row['ingredients'] + " " + row['directions']
    df_dataset.at[i, 'text_embeddings'] = get_embedding(text)

In [59]:
# I'll add more fields for filtering (section 2)
import re 

def convert_to_minutes(time_str):
    if time_str is None:
        return None
    days = 0
    hours = 0
    minutes = 0
    
    # Extract hours and minutes using regex
    days_match = re.search(r'(\d+)\s*days?', time_str)
    hrs_match = re.search(r'(\d+)\s*hrs?', time_str)
    mins_match = re.search(r'(\d+)\s*mins?', time_str)
    
    if days_match:
        days = int(days_match.group(1)) * 60 * 24
    if hrs_match:
        hours = int(hrs_match.group(1)) * 60
    if mins_match:
        minutes = int(mins_match.group(1))
    
    return days + hours + minutes


df_dataset['prep_time_minutes'] = df_dataset['prep_time'].apply(convert_to_minutes).fillna(0)
df_dataset['cook_time_minutes'] = df_dataset['cook_time'].apply(convert_to_minutes).fillna(0)
df_dataset['total_time_minutes'] = df_dataset['total_time'].apply(convert_to_minutes).fillna(0)

### 1.2 Document modelling

In [60]:
from typing import List, Optional
from pydantic import BaseModel, ValidationError

class Recipe(BaseModel):
    _id: int 
    recipe_name: str
    prep_time: Optional[str] = None
    cook_time: Optional[str] = None
    total_time: Optional[str] = None
    servings: int
    yield_amt: Optional[str] = None
    ingredients: str
    directions: str
    rating: float
    url: str
    cuisine_path: str
    nutrition: str
    timing: str
    img_src: str
    prep_time_minutes: int
    cook_time_minutes: int
    total_time_minutes: int
    text_embeddings: List[float]

In [61]:
try:
  # Convert each dictionary to a Movie instance
  recipes = [Recipe(**record).dict() for record in df_dataset.to_dict(orient='records')]
  # Get an overview of a single datapoint
  print(recipes[0].keys())
except ValidationError as e:
  print(e)

dict_keys(['recipe_name', 'prep_time', 'cook_time', 'total_time', 'servings', 'yield_amt', 'ingredients', 'directions', 'rating', 'url', 'cuisine_path', 'nutrition', 'timing', 'img_src', 'prep_time_minutes', 'cook_time_minutes', 'total_time_minutes', 'text_embeddings'])


### 1.3 Database creation and connection

In [11]:
from pymongo.mongo_client import MongoClient
from pymongo.operations import SearchIndexModel

In [12]:
def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""

    # gateway to interacting with a MongoDB database cluster
    client = MongoClient(mongo_uri, appname="recipes.demo")
    print("Connection to MongoDB successful")
    return client

if not MONGO_URI:
    print("MONGO_URI not set in environment variables")


In [13]:
database_name = "recipes_dataset"
collection_name = "recipes"

mongo_client = get_mongo_client(MONGO_URI)

# Pymongo client of database and collection
db = mongo_client.get_database(database_name)
collection = db.get_collection(collection_name)

Connection to MongoDB successful


In [14]:
mongo_client.server_info()

{'version': '7.0.12',
 'gitVersion': 'b6513ce0781db6818e24619e8a461eae90bc94fc',
 'modules': ['enterprise'],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 12, 0],
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 'storageEngines': ['devnull', 'inMemory', 'queryable_wt', 'wiredTiger'],
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1725816252, 14),
  'signature': {'hash': b'P\x88\xc6\x91\x13\x07@6{{a\xe3Z\x1f\x0b{\x19a\x99\xf1',
   'keyId': 7366710683280343048}},
 'operationTime': Timestamp(1725816252, 14)}

In [15]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 961, 'electionId': ObjectId('7fffffff000000000000010b'), 'opTime': {'ts': Timestamp(1725816254, 131), 't': 267}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725816254, 131), 'signature': {'hash': b"'@\xc6\x17^%\xdc\xab\xb9\xf6@\x9a\xae\xb3\xbemE\x19\x11\xf2", 'keyId': 7366710683280343048}}, 'operationTime': Timestamp(1725816254, 131)}, acknowledged=True)

### 1.4 Data ingestion

In [16]:
# The ingestion process might take a few minutes
collection.insert_many(recipes)
print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


### 1.5 Vector search index definition

In [17]:
text_embedding_field_name = "text_embeddings"
# MongoDB Atlas Vector Search index name
vector_search_index_name_text = "vector_index_text"

In [18]:
vector_search_index_model = SearchIndexModel(
    definition={
        "mappings": { # describes how fields in the database documents are indexed and stored
            "dynamic": True, # automatically index new fields that appear in the document
            "fields": { # properties of the fields that will be indexed.
                text_embedding_field_name: { 
                    "dimensions": 768, # size of the vector.
                    "similarity": "cosine", # algorithm used to compute the similarity between vectors
                    "type": "knnVector",
                }
            },
        }
    },
    name=vector_search_index_name_text, # identifier for the vector search index
)

In [19]:
# Check if the index already exists
index_exists = False
for index in collection.list_indexes():
    print(index)
    if index['name'] == vector_search_index_name_text:
        index_exists = True
        break

import time

# Create the index if it doesn't exist
if not index_exists:
    try:
        result = collection.create_search_index(model=vector_search_index_model)
        print("Creating index...")
        time.sleep(20)  # Sleep for 20 seconds, adding sleep to ensure vector index has compeleted inital sync before utilization
        print("Index created successfully:", result)
        print("Wait a few minutes before conducting search with index to ensure index intialization")
    except Exception as e:
        print(f"Error creating vector search index: {str(e)}")
else:
    print(f"Index '{vector_search_index_name_text}' already exists.")

# NOTE: if the output of this process is Error creating vector search index: Duplicate Index, you may proceed to the next cell if you intend to still use a previously created index

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
Error creating vector search index: Duplicate Index, full error: {'ok': 0.0, 'errmsg': 'Duplicate Index', 'code': 68, 'codeName': 'IndexAlreadyExists', '$clusterTime': {'clusterTime': Timestamp(1725816265, 2), 'signature': {'hash': b'\x04\xbc+-;\x06#a\x88\xd5\x1a\x96,Q\xbaw(F\x19\xfa', 'keyId': 7366710683280343048}}, 'operationTime': Timestamp(1725816265, 2)}


In [20]:
#collection.create_search_index(model=vector_search_index_model)

### 1.6 Compose Vector Search Query

In [21]:
def vector_search(user_query, db, collection, vector_index='vector_index_text'):

    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        raise Exception ("embedding generation failed")
    
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,
            "queryVector": query_embedding,
            "path": text_embedding_field_name,
            "numCandidates": 100,
            "limit": 20 
        }
    }

    pipeline = [vector_search_stage]

    results = collection.aggregate(pipeline)

    explain_query_execution = db.command(
        'explain', {
            'aggregate': collection.name,
            'pipeline': pipeline,
            'cursor': {}
        },
        verbosity = 'executionStats'
    )

    vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
    millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

    print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")

    return list(results)


In [22]:
query_embedding = get_embedding('Apple Pie')

if query_embedding is None:
    raise Exception ("embedding generation failed")

vector_search_stage = {
    "$vectorSearch": {
        "index": 'vector_index_text',
        "queryVector": query_embedding,
        "path": text_embedding_field_name,
        "numCandidates": 100,
        "limit": 20 
    }
}

pipeline = [vector_search_stage]

results = collection.aggregate(pipeline)

explain_query_execution = db.command(
    'explain', {
        'aggregate': collection.name,
        'pipeline': pipeline,
        'cursor': {}
    },
    verbosity = 'executionStats'
)

vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")



Total time for the execution to complete on the database server: 0.075253 milliseconds


### 1.7 Handling User Query

In [88]:
class SearchResultItem(BaseModel):
    recipe_name: str
    total_time: Optional[str] = None
    servigs: Optional[str] = None
    ingredients: str
    directions: str
    rating: float

In [24]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="Qwen/Qwen2-7B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm a digital friend, here to blend,\nIn your queries, I'll do my best to bend.\nFrom poetry to prose, or facts to fun,\nJust ask away, and I'll reply, in rhymes, like the sun.", role='assistant', function_call=None, tool_calls=None)


In [25]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection):

    get_knowledge = vector_search(query, db, collection)

    # Check if there are any results
    if not get_knowledge:
        return "No results found.", "No source information available."
        
     # Convert search results into a list of SearchResultItem models
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # Convert search results into a DataFrame for better rendering in Jupyter
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Generate system response using OpenAI's completion
    completion = client.chat.completions.create(
        model="Qwen/Qwen2-7B-Instruct-GGUF",
        messages=[
            {
                "role": "system", 
                "content": "You are a recipes recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response, and Source Information
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Display the DataFrame as an HTML table
    display(HTML(search_results_df.to_html()))

    # Return structured response and source info as a string
    return system_response

In [26]:
handle_user_query('Empanada', db, collection)

Total time for the execution to complete on the database server: 0.081529 milliseconds
- User Question:
Empanada

- System Response:
Based on your query, it seems you're interested in a recipe for an empanada-like dish. However, I noticed that there are no direct results matching "empanada" from the list provided. The closest matches seem to be for dishes such as tacos and quesadillas which have similar characteristics.

For a Carne Asada Taco (Recipe ID 3), you can consider these ingredients:

- 1 ½ pounds boneless beef top sirloin, cut into strips
- For the carne asada marinade: 
    - 3 cloves garlic, minced  
    - 2 tablespoons cumin
    - 1 tablespoon dried oregano
    - 1 tablespoon smoked paprika
    - 1 teaspoon salt
    - Juice of half a lemon or lime
    - Olive oil

For the taco shell, you can either make your own or purchase pre-made shells. You will also need:
- Shredded cheese (such as cheddar)
- Fresh toppings such as diced tomatoes, sliced onions, and chopped cilantro.

,recipe_name,total_time,servigs,ingredients,directions
0,Honduran Baleadas,47 mins,None,"2 cups all-purpose flour, 1 cup water, ½ cup vegetable oil, 1 egg, ½ teaspoon salt","Mix flour, water, vegetable oil, egg, and salt in a large bowl; knead until dough is smooth and no longer sticky.\nForm the dough into 8 golf ball-sized balls. Cover and let rest, about 20 minutes.\nStretch each ball of dough into a thick tortilla.\nHeat a large skillet over medium-high heat. Cook each tortilla until browned and lightly puffed, about 1 minute per side.\nLayer refried beans, avocado, and queso fresco over tortillas. Drizzle crema on top; fold tortillas in half over filling."
1,Black Bean Avocado Salsa,2 hrs 15 mins,None,"1 (15 ounce) can black beans (such as Bush's®), rinsed and drained, 1 (11 ounce) can whole kernel sweet corn, drained, 4 roma (plum) tomatoes, seeded and chopped, 1 small red bell pepper, diced, 1 jalapeno pepper, seeded and minced, ⅓ cup chopped fresh cilantro, ¼ cup diced red onion, ¼ cup fresh lime juice, 2 tablespoons red wine vinegar, 1 teaspoon salt, ½ teaspoon ground black pepper, 2 avocados, diced","Mix black beans, corn, tomatoes, red bell pepper, jalapeno pepper, cilantro, red onion, lime juice, vinegar, salt, and black pepper in a bowl; fold avocado into the mixture. Cover bowl with plastic wrap, putting it right on top of salsa; chill at least 2 hours."
2,Caribbean Fish With Mango Salsa,1 hrs 25 mins,None,"1 tablespoon paprika, 2 teaspoons curry powder, 2 teaspoons ground cumin, 1 ½ teaspoons ground allspice, 1 teaspoon ground ginger, 1 teaspoon ground coriander, ¾ teaspoon salt, ½ teaspoon freshly ground black pepper, ¼ teaspoon ground fennel seed (Optional), ⅛ teaspoon cayenne pepper (Optional)","Mix together paprika, curry powder, cumin, allspice, ginger, coriander, salt, black pepper, fennel, and cayenne pepper in a bowl; set aside.\nIn a bowl, lightly toss mango, pineapple, red bell pepper, black beans, red onion, and cilantro in a bowl; pour lime juice over mango mixture and toss again. Cover the bowl and refrigerate until chilled, at least 30 minutes.\nWhisk together egg and milk in a bowl. In a separate shallow bowl, stir panko crumbs with coconut. Stir about 1 tablespoon of the spice mix, or to taste, into the panko crumb mixture.\nHeat olive oil in a skillet over medium heat. Dip tilapia fillets into egg mixture, then press gently into panko crumb mixture to coat both sides of fillets. Brush off any loose crumbs, then lay fillets into the hot oil. Pan-fry until fish is opaque inside and golden brown outside, 3 to 5 minutes per side. Serve with mango salsa."
3,Carne Asada Tacos,1 hrs 15 mins,None,"1 ½ pounds boneless beef top sirloin, cut into thin bite-size slices, ½ teaspoon salt, 1 teaspoon freshly ground black pepper, crushed red pepper to taste, 1 lime, 1 (28 ounce) can tomatillos, 2 fresh jalapeno peppers, seeded, 4 tablespoons canola oil, divided, 1 (10.5 ounce) can beef broth, 12 (6 inch) corn tortillas, ½ large onion, chopped, 2 tomatoes, chopped, 1 avocado - peeled, pitted and sliced, 1 bunch fresh cilantro, chopped, 1 lemon","Place sliced meat into a shallow bowl, and season with salt, black pepper, and crushed red pepper. Squeeze the lime juice over the meat, and turn until evenly coated. Cover, and refrigerate for 30 minutes.\nIn a blender or food processor, combine tomatillo and jalapeno. Puree for 15 to 20 seconds, or until thick. Heat 1 tablespoon oil in a large skillet over medium high heat. Carefully pour in tomatillo mixture. Cook, stirring frequently, for 5 minutes. Stir in beef broth. Reduce heat, and simmer for 20 to 30 minutes, or until mixture coats a spoon. Transfer mixture to a serving dish.\nHeat tablespoon oil in a large skillet over high heat. Stir in 1/3 of the beef, and saute for 1 minute. Transfer to serving dish. Repeat with remaining beef. Meanwhile, heat tortillas in the oven or microwave, according to package instructions.\nTo serve, place two tortillas on top of each

'Based on your query, it seems you\'re interested in a recipe for an empanada-like dish. However, I noticed that there are no direct results matching "empanada" from the list provided. The closest matches seem to be for dishes such as tacos and quesadillas which have similar characteristics.\n\nFor a Carne Asada Taco (Recipe ID 3), you can consider these ingredients:\n\n- 1 ½ pounds boneless beef top sirloin, cut into strips\n- For the carne asada marinade: \n    - 3 cloves garlic, minced  \n    - 2 tablespoons cumin\n    - 1 tablespoon dried oregano\n    - 1 tablespoon smoked paprika\n    - 1 teaspoon salt\n    - Juice of half a lemon or lime\n    - Olive oil\n\nFor the taco shell, you can either make your own or purchase pre-made shells. You will also need:\n- Shredded cheese (such as cheddar)\n- Fresh toppings such as diced tomatoes, sliced onions, and chopped cilantro.\n- Salsa, guacamole, and sour cream for serving.\n\nDirections:\n1. Mix the marinade ingredients together in a bow

## 2. filtering 
Process additional time have only numeric values, then do a filter on it  
A filter can also be done on servings

### 2.1 Handling additional stages
Modifiy `vector_search` and `handle_user_query` to handle additional stages

In [27]:
def vector_search(user_query, db, collection, additional_stages=[], vector_index='vector_index_text'):

    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        raise Exception ("embedding generation failed")
    
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,
            "queryVector": query_embedding,
            "path": text_embedding_field_name,
            "numCandidates": 100,
            "limit": 20 
        }
    }

    pipeline = [vector_search_stage] + additional_stages

    results = collection.aggregate(pipeline)

    explain_query_execution = db.command(
        'explain', {
            'aggregate': collection.name,
            'pipeline': pipeline,
            'cursor': {}
        },
        verbosity = 'executionStats'
    )

    vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
    millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

    print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")

    return list(results)


In [28]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Assuming vector_search returns a list of dictionaries with keys 'title' and 'plot'
    get_knowledge = vector_search(query, db, collection, stages, vector_index)

    # Check if there are any results
    if not get_knowledge:
        return "No results found.", "No source information available."

    # Convert search results into a list of SearchResultItem models
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # Convert search results into a DataFrame for better rendering in Jupyter
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Generate system response using OpenAI's completion
    completion = client.chat.completions.create(
        model="Qwen/Qwen2-7B-Instruct-GGUF",
        messages=[
            {
                "role": "system", 
                "content": "You are a recipes recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response, and Source Information
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Display the DataFrame as an HTML table
    display(HTML(search_results_df.to_html()))

    # Return structured response and source info as a string
    return system_response

### 2.2 Adding a post filter operator
filters to be added: 
* **Lemon** in ingredients
* servings: more than 2 and less than 10
* total time: less than 60 minutes

In [32]:

# Specifying the metadata field to limit documents on
search_path = "ingredients"

# Create a match stage
match_stage = {
    "$match": {
       search_path: re.compile(r"lemon", re.IGNORECASE),
       "servings": { "$gt": 2},
       "total_time_minutes": { "$lt": 60}
    }
}

additional_stages = [match_stage]

In [33]:
query = '''I want to eat something sweet and tart. 
Can you recommend something for a small get together?
'''

handle_user_query(query, db, collection, additional_stages)

Total time for the execution to complete on the database server: 0.079814 milliseconds
- User Question:
I want to eat something sweet and tart. 
Can you recommend something for a small get together?


- System Response:
Certainly! Based on your preference for something sweet and tart, I would recommend the Summer Fruit Salad with Whipped Cream. This dish combines fresh summer fruits like strawberries, blueberries, and raspberries with a light whipped cream topping to create a delightful balance of sweetness and tartness.

**Recipe Name:** Summer Fruit Salad with Whipped Cream

**Total Time:** 40 minutes

**Servings:** This recipe doesn't specify the number of servings, but you can adjust it based on your gathering size. It's ideal for small get-togethers as it typically serves a few people.

### Ingredients:

- **3/4 cup water**
- **2 tablespoons white sugar**
- **1 cup raspberries (fresh or frozen), thawed**
- **2 cups blueberries (fresh or frozen), thawed**
- **6 large strawberries, 

,recipe_name,total_time,servigs,ingredients,directions
0,Summer Fruit Salad with Whipped Cream,40 mins,None,"⅓ cup water, 5 tablespoons white sugar, ¼ cup raspberry-flavored liqueur (such as Chambord®), 2 tablespoons fresh lemon juice","Stir water, 5 tablespoons sugar, 1/4 cup raspberry-flavored liqueur, and lemon juice together in a saucepan over medium-low heat. Bring the mixture to a boil, reduce heat to low, and simmer, stirring frequently, until the sugar dissolves completely, about 8 minutes. Set aside to cool.\nMix nectarines, cherries, strawberries, raspberries, and blueberries in a large bowl. Pour the cooled sugar mixture over the fruit and toss gently to coat.\nBeat heavy whipping cream and 2 teaspoons sugar in a bowl with an electric hand mixer on high until stiff peaks form. Fold 1 teaspoon raspberry-flavored liqueur into the whipped cream; spoon over the fruit salad and sprinkle evenly with almonds."
1,Watermelon Goat Cheese Salad,10 mins,None,"¼ cup extra-virgin olive oil, 2 tablespoons lemon juice, 2 tablespoons red wine vinegar, ¼ teaspoon salt, Ground pepper, to taste, 4 cups spring mix, 4 cups seedless watermelon, diced, ½ (4 ounce) log Montchevre® honey goat cheese , ½ cup hazelnuts, chopped","Whisk together olive oil, lemon juice, vinegar, salt, and pepper.\nIn a large bowl, gently toss spring mix and watermelon together with vinaigrette.\nTop with goat cheese and hazelnuts to serve."
2,Old-Fashioned Lemonade,10 mins,None,"6 lemons, 1 cup white sugar, 6 cups water, or more as needed","Juice lemons; you should have 1 cup juice.\nCombine juice, sugar, and water in a 1/2-gallon pitcher. Stir until sugar dissolves. Taste and add more water if desired.\nChill and serve over ice.\n\n\n\n\n\n\n\n\n\n\n\nDOTDASH MEREDITH FOOD STUDIOS\n"
3,Fabulous Fruit Salad,20 mins,None,"1 red apple, cored and chopped, 1 Granny Smith apple, cored and chopped, 1 nectarine, pitted and sliced, 2 stalks celery, chopped, ½ cup dried cranberries, ½ cup chopped walnuts, 1 (8 ounce) container nonfat lemon yogurt","In a large bowl, combine red apple, Granny Smith apple, nectarine, celery, dried cranberries, and walnuts. Mix in yogurt. Chill until ready to serve."


"Certainly! Based on your preference for something sweet and tart, I would recommend the Summer Fruit Salad with Whipped Cream. This dish combines fresh summer fruits like strawberries, blueberries, and raspberries with a light whipped cream topping to create a delightful balance of sweetness and tartness.\n\n**Recipe Name:** Summer Fruit Salad with Whipped Cream\n\n**Total Time:** 40 minutes\n\n**Servings:** This recipe doesn't specify the number of servings, but you can adjust it based on your gathering size. It's ideal for small get-togethers as it typically serves a few people.\n\n### Ingredients:\n\n- **3/4 cup water**\n- **2 tablespoons white sugar**\n- **1 cup raspberries (fresh or frozen), thawed**\n- **2 cups blueberries (fresh or frozen), thawed**\n- **6 large strawberries, hulled and sliced**\n- **2 cups fresh pineapple chunks**\n- **1 cup mango cubes**\n\n### Directions:\n\n1. Stir together water, 2 tablespoons sugar, and raspberries in a medium saucepan over medium heat.\n

### 2.3 Vector search with PreFilter

In [35]:
vector_index_with_filter = "vector_index_with_filter"

new_vector_search_index_model = SearchIndexModel(
    definition={
        "mappings": { 
            "dynamic": True, 
            "fields": { 
                "text_embeddings": { 
                    "dimensions": 768, 
                    "similarity": "cosine", 
                    "type": "knnVector",
                },
                "servings" : {
                    "type": "number"
                },
                "total_time_minutes": {
                    "type": "number"
                }
            },
        }
    },
    name=vector_index_with_filter, 
)

In [36]:
# Create the new index
try:
    result = collection.create_search_index(model=new_vector_search_index_model)
    print("Creating index...")
    time.sleep(20)  # Sleep for 20 seconds, adding sleep to ensure vector index has compeleted inital sync before utilization
    print("New index created successfully:", result)
except Exception as e:
    print(f"Error creating new vector search index: {str(e)}")

Creating index...
New index created successfully: vector_index_with_filter


In [78]:
def vector_search(user_query, db, collection, additional_stages=[], vector_index="vector_index_text"):
    query_embedding = get_embedding(user_query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index, 
            "queryVector": query_embedding, 
            "path": "text_embeddings", 
            "numCandidates": 150, 
            "limit": 20, 
            "filter": {
                "$and": [
                    {"servings": {"$gte": 2.0}}, 
                    {"total_time_minutes": {"$lte": 60.0}},
                ]
            },
        }
    }

    pipeline = [vector_search_stage] + additional_stages

    results = collection.aggregate(pipeline)
    explain_query_execution = db.command( # sends a database command directly to the MongoDB server
        'explain', { # return information about how MongoDB executes a query or command without actually running it
            'aggregate': collection.name, # specifies the name of the collection on which the aggregation is performed
            'pipeline': pipeline, # the aggregation pipeline to analyze
            'cursor': {} # indicates that default cursor behavior should be used
        }, 
        verbosity='executionStats') # detailed statistics about the execution of each stage of the aggregation pipeline

    vector_search_explain = explain_query_execution['stages'][0]['$vectorSearch']
    millis_elapsed = vector_search_explain['explain']['collectStats']['millisElapsed']

    print(f"Total time for the execution to complete on the database server: {millis_elapsed} milliseconds")
    return list(results)

In [79]:
query = '''I want to eat something salty for snacking.
Can you recommend something for a small get together?
'''

handle_user_query(query, db, collection, vector_index=vector_index_with_filter)

Total time for the execution to complete on the database server: 0.081482 milliseconds
- User Question:
I want to eat something salty for snacking.
Can you recommend something for a small get together?


- System Response:
Here are some salty snack recipes that would be perfect for a small get-together:

1. **Pear-Fig Salad** (20 mins): This salad combines savory and sweet flavors, making it an excellent choice for snacking. It's a mix of romaine lettuce, fresh figs, pears, and pecans.

2. **Salsa Salad** (30 mins): A refreshing option that's both salty and satisfying. Combine tomatoes cut into bite-size pieces with other vegetables or fruits, then dress with olive oil and lemon juice for an easy-to-make dish.

Both of these recipes will provide a good balance of saltiness and freshness, making them great choices for a snack during your gathering!



,recipe_name,total_time,servigs,ingredients,directions
0,Strawberry Mango Mesclun Salad,15 mins,None,"½ cup sugar, ¾ cup canola oil, 1 teaspoon salt, ¼ cup balsamic vinegar, 8 cups mixed salad greens, 2 cups sweetened dried cranberries, ½ pound fresh strawberries, quartered, 1 mango - peeled, seeded, and cubed, ½ cup chopped onion, 1 cup slivered almonds","Place sugar, oil, salt, and vinegar in a jar with a lid. Seal the jar and shake vigorously to mix.\nIn a large bowl, mix salad greens, sweetened dried cranberries, strawberries, mango, and onion. To serve, toss with dressing and sprinkle with almonds."
1,Pomegranate Holiday Cocktail,5 mins,None,"3 cups ice, or as needed, 2 fluid ounces vodka, 2 fluid ounces pomegranate juice, 1 lime, juiced, ½ fluid ounce orange-flavored liqueur, 2 ½ fluid ounces club soda, or as needed, 1 teaspoon pomegranate seeds, or to taste","Fill 2 highball glasses and a cocktail shaker with ice.\nAdd vodka, pomegranate juice, lime juice, and orange-flavored liqueur to the cocktail shaker and secure the lid. Place one hand on the lid and one hand on the shaker; shake vigorously until the outside of the shaker frosts, about a slow count to 10.\nPour evenly into the prepared glasses. Top with soda and pomegranate seeds; stir before serving."
2,Waldorf Salad,20 mins,None,"½ cup mayonnaise, 1 tablespoon white sugar, 1 teaspoon lemon juice, ⅛ teaspoon salt, 3 apples -- peeled, cored, and chopped, 1 cup thinly sliced celery, ½ cup chopped walnuts, ½ cup raisins (Optional)","Whisk together mayonnaise, sugar, lemon juice, and salt in a serving bowl.\nStir in apples, celery, walnuts, and raisins. Cover and chill in the refrigerator until ready to serve."
3,Cranberry Pear Salad,15 mins,None,"¾ cup olive oil, ¼ cup balsamic vinegar, 1 tablespoon dark brown sugar, 2 ½ teaspoons minced garlic, ½ teaspoon salt, ½ teaspoon freshly ground black pepper, 6 cups mixed baby greens, 3 pears, thinly sliced, 1 cup dried cranberries, 1 cup toasted pecans, ¾ cup cubed Havarti cheese, ½ cup toasted sliced almonds","Whisk olive oil, balsamic vinegar, brown sugar, garlic, salt, and black pepper in a medium bowl.\nCombine greens, pears, cranberries, pecans, and Havarti cheese in a large bowl. Pour balsamic vinegar dressing over salad and toss. Garnish with almonds to serve."
4,"Strawberry, Kiwi, and Spinach Salad",15 mins,None,"2 tablespoons raspberry vinegar, 2 ½ tablespoons raspberry jam, ⅓ cup vegetable oil, 8 cups spinach, rinsed and torn into bite-size pieces, ½ cup chopped walnuts, 8 strawberries, quartered, 2 kiwis, peeled and sliced","Mix together raspberry vinegar, raspberry jam, and vegetable oil in a small container.\nCombine spinach, nuts, strawberries, and kiwi in a salad bowl. Toss with raspberry dressing."
5,Pear-Fig Salad,20 mins,None,"2 cups torn romaine lettuce, 6 fresh figs, quartered, 1 large ripe but firm pear, peeled and thinly sliced, ¼ cup toasted pecans, chopped, ¼ cup shredded Gruyere cheese, 1 small red onion, thinly sliced, 3 tablespoons extra-virgin olive oil, 3 tablespoons balsamic vinegar, salt and freshly ground black pepper to taste","Combine romaine lettuce, figs, pear, pecans, Gruyere cheese, and onion in a large bowl. Drizzle evenly with olive oil and vinegar. Season with salt and pepper. Toss lightly and serve."
6,Pomegranate Molasses Barbecue Sauce,15 mins,None,"1 ½ tablespoons dark soy sauce, 2 tablespoons ketchup, 2 tablespoons pomegranate molasses, 2 tablespoons honey, 1 tablespoon sunflower seed oil, 1 teaspoon smooth mustard, 1 clove garlic, minced, ¼ teaspoon sea salt, ¼ teaspoon coarsely ground black pepper, ¼ teaspoon ground paprika","In a bowl, stir together the soy sauce, ketchup, pomegranate molasses, honey, sunflower seed oil, mustard, garlic, sea salt, black pepper, and paprika to make a smooth paste."
7,Pomegranate Relish,10 mins,None,"2 pomegranates, 3 tablespoons olive oil, 1 shallot, minced, 1 tablespoon lime juice, salt and pepper to taste, ½ cup fresh cilantro, chopped","Jui

"Here are some salty snack recipes that would be perfect for a small get-together:\n\n1. **Pear-Fig Salad** (20 mins): This salad combines savory and sweet flavors, making it an excellent choice for snacking. It's a mix of romaine lettuce, fresh figs, pears, and pecans.\n\n2. **Salsa Salad** (30 mins): A refreshing option that's both salty and satisfying. Combine tomatoes cut into bite-size pieces with other vegetables or fruits, then dress with olive oil and lemon juice for an easy-to-make dish.\n\nBoth of these recipes will provide a good balance of saltiness and freshness, making them great choices for a snack during your gathering!"

## 3. Adding a projection stage

In [80]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Assuming vector_search returns a list of dictionaries with keys 'title' and 'plot'
    get_knowledge = vector_search(query, db, collection, stages, vector_index)

    print("List of all fields of the first document, before model conformance")
    print(get_knowledge[0].keys())

    # Check if there are any results
    if not get_knowledge:
        return "No results found.", "No source information available."

    # Convert search results into a list of SearchResultItem models
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # Convert search results into a DataFrame for better rendering in Jupyter
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Generate system response using OpenAI's completion
    completion = client.chat.completions.create(
        model="Qwen/Qwen2-7B-Instruct-GGUF",
        messages=[
            {
                "role": "system", 
                "content": "You are a recipes recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response, and Source Information
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Display the DataFrame as an HTML table
    display(HTML(search_results_df.to_html()))

    # Return structured response and source info as a string
    return system_response

In [86]:
projection_stage = {
    "$project": {
        "_id": 0,
        "recipe_name": 1,  
        "total_time": 1,
        "servigs": 1,
        "ingredients": 1,
        "directions": 1, 
        "rating": 1
    }
}

additional_stages = [projection_stage]

In [89]:
query = '''I want to eat something salty for snacking.
Can you recommend something for a small get together?
'''

handle_user_query(query, db, collection, additional_stages, vector_index=vector_index_with_filter)

Total time for the execution to complete on the database server: 0.077633 milliseconds
List of all fields of the first document, before model conformance
dict_keys(['recipe_name', 'total_time', 'ingredients', 'directions', 'rating'])
- User Question:
I want to eat something salty for snacking.
Can you recommend something for a small get together?


- System Response:
Based on your preference for something salty to snack on and your request for recommendations suitable for a small get-together, I suggest trying out the "Date-Nut Balls" recipe or the "Peanut Butter Mango Smoothie". Both of these recipes are easy to prepare and would be great options for snacking.

Here's a brief overview of each:

### Date-Nut Balls
- **Ingredients**: Butter, pitted chopped dates, ground nuts (like almonds), cinnamon.
- **Directions**:
  1. Melt butter in a saucepan over medium heat; cook until melted.
  2. Add the pitted chopped dates to the saucepan and cook, stirring constantly, for about 5 minutes or

,recipe_name,total_time,servigs,ingredients,directions,rating
0,Strawberry Mango Mesclun Salad,15 mins,None,"½ cup sugar, ¾ cup canola oil, 1 teaspoon salt, ¼ cup balsamic vinegar, 8 cups mixed salad greens, 2 cups sweetened dried cranberries, ½ pound fresh strawberries, quartered, 1 mango - peeled, seeded, and cubed, ½ cup chopped onion, 1 cup slivered almonds","Place sugar, oil, salt, and vinegar in a jar with a lid. Seal the jar and shake vigorously to mix.\nIn a large bowl, mix salad greens, sweetened dried cranberries, strawberries, mango, and onion. To serve, toss with dressing and sprinkle with almonds.",4.8
1,Pomegranate Holiday Cocktail,5 mins,None,"3 cups ice, or as needed, 2 fluid ounces vodka, 2 fluid ounces pomegranate juice, 1 lime, juiced, ½ fluid ounce orange-flavored liqueur, 2 ½ fluid ounces club soda, or as needed, 1 teaspoon pomegranate seeds, or to taste","Fill 2 highball glasses and a cocktail shaker with ice.\nAdd vodka, pomegranate juice, lime juice, and orange-flavored liqueur to the cocktail shaker and secure the lid. Place one hand on the lid and one hand on the shaker; shake vigorously until the outside of the shaker frosts, about a slow count to 10.\nPour evenly into the prepared glasses. Top with soda and pomegranate seeds; stir before serving.",5.0
2,Waldorf Salad,20 mins,None,"½ cup mayonnaise, 1 tablespoon white sugar, 1 teaspoon lemon juice, ⅛ teaspoon salt, 3 apples -- peeled, cored, and chopped, 1 cup thinly sliced celery, ½ cup chopped walnuts, ½ cup raisins (Optional)","Whisk together mayonnaise, sugar, lemon juice, and salt in a serving bowl.\nStir in apples, celery, walnuts, and raisins. Cover and chill in the refrigerator until ready to serve.",4.6
3,Cranberry Pear Salad,15 mins,None,"¾ cup olive oil, ¼ cup balsamic vinegar, 1 tablespoon dark brown sugar, 2 ½ teaspoons minced garlic, ½ teaspoon salt, ½ teaspoon freshly ground black pepper, 6 cups mixed baby greens, 3 pears, thinly sliced, 1 cup dried cranberries, 1 cup toasted pecans, ¾ cup cubed Havarti cheese, ½ cup toasted sliced almonds","Whisk olive oil, balsamic vinegar, brown sugar, garlic, salt, and black pepper in a medium bowl.\nCombine greens, pears, cranberries, pecans, and Havarti cheese in a large bowl. Pour balsamic vinegar dressing over salad and toss. Garnish with almonds to serve.",4.8
4,"Strawberry, Kiwi, and Spinach Salad",15 mins,None,"2 tablespoons raspberry vinegar, 2 ½ tablespoons raspberry jam, ⅓ cup vegetable oil, 8 cups spinach, rinsed and torn into bite-size pieces, ½ cup chopped walnuts, 8 strawberries, quartered, 2 kiwis, peeled and sliced","Mix together raspberry vinegar, raspberry jam, and vegetable oil in a small container.\nCombine spinach, nuts, strawberries, and kiwi in a salad bowl. Toss with raspberry dressing.",4.7
5,Pear-Fig Salad,20 mins,None,"2 cups torn romaine lettuce, 6 fresh figs, quartered, 1 large ripe but firm pear, peeled and thinly sliced, ¼ cup toasted pecans, chopped, ¼ cup shredded Gruyere cheese, 1 small red onion, thinly sliced, 3 tablespoons extra-virgin olive oil, 3 tablespoons balsamic vinegar, salt and freshly ground black pepper to taste","Combine romaine lettuce, figs, pear, pecans, Gruyere cheese, and onion in a large bowl. Drizzle evenly with olive oil and vinegar. Season with salt and pepper. Toss lightly and serve.",4.0
6,Pomegranate Molasses Barbecue Sauce,15 mins,None,"1 ½ tablespoons dark soy sauce, 2 tablespoons ketchup, 2 tablespoons pomegranate molasses, 2 tablespoons honey, 1 tablespoon sunflower seed oil, 1 teaspoon smooth mustard, 1 clove garlic, minced, ¼ teaspoon sea salt, ¼ teaspoon coarsely ground black pepper, ¼ teaspoon ground paprika","In a bowl, stir together the soy sauce, ketchup, pomegranate molasses, honey, sunflower seed oil, mustard, garlic, sea salt, black pepper, and paprika to make a smooth paste.",4.8
7,Pomegranate Relish,10 mins,None,"2 pomegranates, 3 tablespoons olive oil, 1 shallot, minced, 1 tablespoon lime juice, salt and pepper to taste, 

'Based on your preference for something salty to snack on and your request for recommendations suitable for a small get-together, I suggest trying out the "Date-Nut Balls" recipe or the "Peanut Butter Mango Smoothie". Both of these recipes are easy to prepare and would be great options for snacking.\n\nHere\'s a brief overview of each:\n\n### Date-Nut Balls\n- **Ingredients**: Butter, pitted chopped dates, ground nuts (like almonds), cinnamon.\n- **Directions**:\n  1. Melt butter in a saucepan over medium heat; cook until melted.\n  2. Add the pitted chopped dates to the saucepan and cook, stirring constantly, for about 5 minutes or until the mixture begins to thicken slightly.\n  3. Remove from heat and let cool slightly.\n  4. Stir in ground nuts and cinnamon into the mixture.\n  5. Chill the mixture for at least an hour.\n  6. Roll chilled mixture into balls.\n\n### Peanut Butter Mango Smoothie\n- **Ingredients**: Vanilla yogurt, banana, mango, peanut butter.\n- **Directions**:\n  1

## 4. Boosting search results

In [91]:
weighting_stage = {
    "$addFields": {
        "combinedScore": {
            "$add": [
                {"$multiply": ["$rating", 0.9]},
                {"$multiply": ["$total_time_minutes", -0.1]}  # rewards short time recipes
            ]
        }
    }
}

In [92]:
# Apply the combinedScore for sorting
sorting_stage_sort = {
    "$sort": {"combinedScore": -1}  # Descending order to boost higher combined scores
}

In [93]:
additional_stages = [projection_stage, weighting_stage, sorting_stage_sort]

In [94]:
query = '''I want to eat something salty for snacking.
Can you recommend something for a small get together?
'''

handle_user_query(query, db, collection, additional_stages, vector_index=vector_index_with_filter)

Total time for the execution to complete on the database server: 0.077498 milliseconds
List of all fields of the first document, before model conformance
dict_keys(['_id', 'recipe_name', 'prep_time', 'cook_time', 'total_time', 'servings', 'yield_amt', 'ingredients', 'directions', 'rating', 'url', 'cuisine_path', 'nutrition', 'timing', 'img_src', 'prep_time_minutes', 'cook_time_minutes', 'total_time_minutes', 'text_embeddings', 'combinedScore'])
- User Question:
I want to eat something salty for snacking.
Can you recommend something for a small get together?


- System Response:
For a small get together with a salty snack preference, I recommend trying the "Pineapple Glaze for Ham" or "Watermelon Goat Cheese Salad." Both dishes are quick to prepare and will add a savory touch to your gathering:

1. **Pineapple Glaze for Ham**: This dish is perfect if you're planning to serve ham as an appetizer. The glaze adds a sweet and salty flavor that pairs well with the ham.

2. **Watermelon Goat 

,recipe_name,total_time,servigs,ingredients,directions,rating
0,Pomegranate Holiday Cocktail,5 mins,None,"3 cups ice, or as needed, 2 fluid ounces vodka, 2 fluid ounces pomegranate juice, 1 lime, juiced, ½ fluid ounce orange-flavored liqueur, 2 ½ fluid ounces club soda, or as needed, 1 teaspoon pomegranate seeds, or to taste","Fill 2 highball glasses and a cocktail shaker with ice.\nAdd vodka, pomegranate juice, lime juice, and orange-flavored liqueur to the cocktail shaker and secure the lid. Place one hand on the lid and one hand on the shaker; shake vigorously until the outside of the shaker frosts, about a slow count to 10.\nPour evenly into the prepared glasses. Top with soda and pomegranate seeds; stir before serving.",5.0
1,Peanut Butter Mango Smoothie,5 mins,None,"1 cup vanilla yogurt, 1 banana, broken into chunks, ½ cup frozen mango chunks, 2 tablespoons peanut butter, or to taste","Blend yogurt, banana, mango, and peanut butter together in a blender until smooth.",4.4
2,Christmas Pomegranate Salad,10 mins,None,"3 cups leafy salad green mix, ½ cup pomegranate seeds, ⅓ cup crumbled blue cheese, ¼ cup crushed walnuts, ¼ cup cranberry vinaigrette","Toss leafy greens, pomegranate seeds, blue cheese, and walnuts together in a medium-sized salad bowl. Add cranberry vinaigrette just before serving.",4.8
3,Pomegranate Relish,10 mins,None,"2 pomegranates, 3 tablespoons olive oil, 1 shallot, minced, 1 tablespoon lime juice, salt and pepper to taste, ½ cup fresh cilantro, chopped","Juice the pomegranates retaining the seeds and juice.\nHeat oil in a small skillet over medium heat. Saute shallots until golden. Stir in pomegranate juice and seeds, lime juice, salt and pepper. Cook for about 3 minutes, or until slightly reduced. Remove from heat and stir in cilantro.\nServe at room temperature.",4.4
4,Strawberry Mango Mesclun Salad,15 mins,None,"½ cup sugar, ¾ cup canola oil, 1 teaspoon salt, ¼ cup balsamic vinegar, 8 cups mixed salad greens, 2 cups sweetened dried cranberries, ½ pound fresh strawberries, quartered, 1 mango - peeled, seeded, and cubed, ½ cup chopped onion, 1 cup slivered almonds","Place sugar, oil, salt, and vinegar in a jar with a lid. Seal the jar and shake vigorously to mix.\nIn a large bowl, mix salad greens, sweetened dried cranberries, strawberries, mango, and onion. To serve, toss with dressing and sprinkle with almonds.",4.8
5,Cranberry Pear Salad,15 mins,None,"¾ cup olive oil, ¼ cup balsamic vinegar, 1 tablespoon dark brown sugar, 2 ½ teaspoons minced garlic, ½ teaspoon salt, ½ teaspoon freshly ground black pepper, 6 cups mixed baby greens, 3 pears, thinly sliced, 1 cup dried cranberries, 1 cup toasted pecans, ¾ cup cubed Havarti cheese, ½ cup toasted sliced almonds","Whisk olive oil, balsamic vinegar, brown sugar, garlic, salt, and black pepper in a medium bowl.\nCombine greens, pears, cranberries, pecans, and Havarti cheese in a large bowl. Pour balsamic vinegar dressing over salad and toss. Garnish with almonds to serve.",4.8
6,Pomegranate Molasses Barbecue Sauce,15 mins,None,"1 ½ tablespoons dark soy sauce, 2 tablespoons ketchup, 2 tablespoons pomegranate molasses, 2 tablespoons honey, 1 tablespoon sunflower seed oil, 1 teaspoon smooth mustard, 1 clove garlic, minced, ¼ teaspoon sea salt, ¼ teaspoon coarsely ground black pepper, ¼ teaspoon ground paprika","In a bowl, stir together the soy sauce, ketchup, pomegranate molasses, honey, sunflower seed oil, mustard, garlic, sea salt, black pepper, and paprika to make a smooth paste.",4.8
7,Mango Relish,15 mins,None,"1 mango - peeled, seeded and diced, 1 teaspoon extra virgin olive oil, ½ red bell pepper, chopped, 2 green onion, thinly sliced, 1 tablespoon chopped cilantro, 1 lime, juiced, ¼ teaspoon salt, 1 pinch cracked black pepper, 1 teaspoon honey","Combine mango, oil, bell pepper, scallions, cilantro, and lime juice in a small bowl. Stir to combine. Season with salt, pepper, and honey. Refrigerate or serve immediately.",4.8
8,"Strawberry, Kiwi, and Spi

'For a small get together with a salty snack preference, I recommend trying the "Pineapple Glaze for Ham" or "Watermelon Goat Cheese Salad." Both dishes are quick to prepare and will add a savory touch to your gathering:\n\n1. **Pineapple Glaze for Ham**: This dish is perfect if you\'re planning to serve ham as an appetizer. The glaze adds a sweet and salty flavor that pairs well with the ham.\n\n2. **Watermelon Goat Cheese Salad**: This salad is refreshing and provides a nice contrast in flavors, offering both saltiness from the cheese and sweetness from the watermelon.\n\nBoth dishes have a quick preparation time of 15 minutes or less, making them ideal for a small get-together where you might not want to spend too much time in the kitchen. Enjoy your snack-making!'

## 5. Prompt Compression

In [105]:
from llmlingua import PromptCompressor
import json

In [106]:
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    model_config={"revision": "main"},
    use_llmlingua2=True,
    device_map="mps",
)


In [107]:
def compress_query_prompt(query):

    demonstration_str = query['demonstration_str']
    instruction = query['instruction']
    question = query['question']

    # 6x Compression
    compressed_prompt = llm_lingua.compress_prompt(
        demonstration_str.split("\n"), 
        instruction=instruction,
        question=question,
        target_token=500,
        rank_method="longllmlingua", 
        context_budget="+100",
        dynamic_context_compression_ratio=0.4,
        reorder_context="sort",
    )

    return json.dumps(compressed_prompt, indent=4)


In [108]:

def handle_user_query_with_compression(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Assuming vector_search returns a list of dictionaries with keys 'title' and 'plot'
    get_knowledge = vector_search(query, db, collection, stages, vector_index)

    # Check if there are any results
    if not get_knowledge:
        return "No results found.", "No source information available."

    # Convert search results into a list of SearchResultItem models
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    # Convert search results into a DataFrame for better rendering in Jupyter
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Prepare information for compression
    query_info = {
        'demonstration_str': search_results_df.to_string(),  # Results from information retrieval process
        'instruction': "Write a high-quality answer for the given question using only the provided search results.",
        'question': query
    }

    # Compress the query prompt using predefined function
    compressed_prompt = compress_query_prompt(query_info)

    # Optional: Print compressed prompts for debugging
    print("Compressed Prompt:\n")
    pprint.pprint(compressed_prompt)
    print("\n" + "=" * 80 + "\n")

    return search_results_df, compressed_prompt

In [112]:
def handle_system_response(query, compressed_prompt):
    # Generate system response using OpenAI's completion
    completion = client.chat.completions.create(
        model="Qwen/Qwen2-7B-Instruct-GGUF",
        messages=[
            {
                "role": "system", 
                "content": "You are a recipes recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{compressed_prompt}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response, and Source Information
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Display the DataFrame as an HTML table
#    display(HTML(search_results_df.to_html()))

    # Return structured response and source info as a string
    return system_response


In [113]:
# Compress the query and get search results
results, compressed_prompt = handle_user_query_with_compression(query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)

Total time for the execution to complete on the database server: 0.077354 milliseconds
Compressed Prompt:

('{\n'
 '    "compressed_prompt": "recipe name time servigs directions\\n\\nChristmas '
 'Pomegranate Salad 10 mins 3 cups \\u00bd seeds crumbled blue cheese crushed '
 'walnuts cranberry vinaigrette leafy greens pomegranate seeds blue cheese '
 'walnuts medium - sized salad bowl cranberry vinaigrette\\n\\nCranberry Pear '
 'Salad 15 mins \\u00be olive oil balsamic vinegar 1 tablespoon brown sugar '
 'teaspoons minced garlic salt black pepper 6 cups baby greens 3 pears sliced '
 'dried cranberries toasted pecans Havarti cheese almonds Whisk olive oil '
 'balsamic vinegar brown sugar garlic salt black pepper medium bowl greens '
 'pears cranberries pecans Havarti cheese bowl balsamic vinegar dressing salad '
 'Garnish almonds\\n\\nPomegranate Molasses Barbecue Sauce 15 mins None '
 '\\u00bd tablespoons dark soy sauce 2 ketchup 2 honey sunflower seed oil 1 '
 'teaspoon smooth mustar

In [114]:
if compressed_prompt:
    # Handle the system response with the compressed prompt
    system_response = handle_system_response(query, compressed_prompt)
else:
    print("No valid results to display.")

- User Question:
I want to eat something salty for snacking.
Can you recommend something for a small get together?


- System Response:
Certainly! Given your preference for salty snacks and the context you provided, which includes several salad recipes, I'd recommend the Cranberry Pear Salad or the Watermelon Goat Cheese Salad for a small get-together. Both of these salads offer a variety of flavors that complement each other well:

### Cranberry Pear Salad
This salad is made with baby greens, pears, dried cranberries, pecans, and Havarti cheese. The dressing is made from olive oil, balsamic vinegar, brown sugar, garlic, salt, and black pepper, which adds a rich flavor to the dish.

### Watermelon Goat Cheese Salad
This refreshing salad includes seedless watermelon, goat cheese (Montchevre\u00ae), and hazelnuts. It's tossed with a vinaigrette made from olive oil, lemon juice, red wine vinegar, salt, and ground pepper.

Both dishes are relatively quick to prepare (around 15-20 minutes) 